In [205]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [206]:
from scipy import stats
from scipy.stats import norm

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [207]:
# Customize matplotlib default settings
matplotlib.rcParams.update({'font.size': 16})
plt.rcParams["figure.figsize"] = (20,10)

In [208]:
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

## 1. Get stock data

In [209]:
! ls ../input

google_stock_data.csv


In [210]:
# read in the stock data
stock_data = pd.read_csv("../input/google_stock_data.csv")

stock_data.head()

,date,open,high,low,close,volume
0,2004-08-19,100.01,104.06,95.96,100.335,44659000.0
1,2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2,2004-08-23,110.76,113.48,109.05,109.400,18256100.0
3,2004-08-24,111.24,111.60,103.57,104.870,15247300.0
4,2004-08-25,104.76,108.00,103.88,106.000,9188600.0


Alpha Vantage metadata for the dataset above:  
{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2019-04-17 16:00:01',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

## 2. Data exploration

In [211]:
stock_data.describe(include='all')

,date,open,high,low,close,volume
count,3691,3691.000000,3691.000000,3691.000000,3691.000000,3.691000e+03
unique,3691,NaN,NaN,NaN,NaN,NaN
top,2011-04-12,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,635.167468,641.086776,628.712554,634.970676,7.387645e+06
std,NaN,259.306949,260.807566,257.595840,259.319778,8.095545e+06
min,NaN,99.090000,101.740000,95.960000,100.010000,3.845900e+04
25%,NaN,469.795000,474.105000,464.680000,469.975000,2.091950e+06
50%,NaN,580.000000,585.000000,574.190000,580.110000,4.645500e+06
75%,NaN,794.975000,799.955000,788.355000,793.485000,9.610600e+06


In [212]:
print("Date range:", stock_data.date.min(), "to", stock_data.date.max())

Date range: 2004-08-19 to 2019-04-17


Let's visualize the various price measures over time.

In [213]:
stock_data.set_index("date", inplace=True)
# stock_data.head()

In [277]:
trace_open = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.open,
                    mode = "lines",
                    name = "open",
                    marker = dict(color = 'blue'),
                    text= stock_data.index)

trace_close = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.close,
                    mode = "lines",
                    name = "close",
                    marker = dict(color = 'green'),
                    text= stock_data.index)

trace_high = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.high,
                    mode = "lines",
                    name = "high",
                    marker = dict(color = 'red'),
                    text= stock_data.index)

trace_low = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.low,
                    mode = "lines",
                    name = "low",
                    marker = dict(color = 'grey'),
                    text= stock_data.index)


layout = dict(title = 'GOOGL stock daily prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Price'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_high, trace_low, trace_open, trace_close], layout = layout)
iplot(fig)

The high, low, open and close prices follow each other pretty closely. We'll pick just one of them, the close price, and focus on it in further analyses. 

Another dimension to the data is the daily trading volume. Let's visualize it together with the close price, so that we can see how both volume and price of Google stock have changed over time. 
In the chart below, the grey circle sizes reflect the relative daily trading volume size. The circles are transparent, so that when they overlap on the chart they form darker grey/black blobs, which highlight periods of consistently high trading volumes.

In [279]:
trace_bubble = go.Scatter(
                            x = stock_data.index,
                            y = stock_data.close,
                            mode = "markers",
                            name = "volume",
                            marker = dict(size = stock_data.volume, 
                                          sizemode='area',
                                          sizeref=2.*max(stock_data.volume)/(30.**2),
                                          sizemin=1,
                                          color = 'black',
                                          opacity = 0.2,
                                          line = {"width": 0}),
                            hovertext  = "close: " + stock_data.close.astype(str) 
                                        + "<br>volume: " + (stock_data.volume/1000000).round(2).astype(str) + "M"
                                        + "<br>date: " + stock_data.index,
                            hoverinfo = 'text'
                        )


layout = dict(title = 'GOOGL stock daily close prices and volumes' 
                    + '<br><br><span style="font-size: 16px; color: darkgrey">Bubble size = Daily volume</span>',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Price'),
              autosize=False,
              width=980,
              height=700
             )
fig = dict(data = [trace_bubble], layout = layout)
iplot(fig)

In the chart above, we can see a bird's-eye view of the Google stock price history. In the early days of Google, its stock traded at relatively low prices and high volumes. In 2008-2009 we can see a large dip in price, likely explained by the Great Recession. After that the stock price quckly recovered, but the trading volume gradually decreased.

Then something unusual happened early in 2014 to the stock's price: it instantly dropped by half. On April 3rd 2014, the Alphabet (GOOGL) stock split in a 1998 to 1000 ratio. After the split, for each 1000 shares of GOOGL stock owned pre-split, a shareholder now owned 1998 shares. This is very close to a 2-to-1 split ratio, so we'll use it as an approximation in the analyses here. So, for each 1 share of Google stock owned pre-split, a shareholder now owned 2 shares. ([Source](https://www.stocksplithistory.com/alphabet/))

The Google stock split was first announced in early 2012 ([source](https://www.fool.com/investing/2017/05/10/alphabet-stock-split-will-the-google-parent-ever-s.aspx)), so the market had plenty of time to adjust and incorporate the news about the split into the stock price. 

Stock splits can be done for a variety of reasons. One popular reason is to keep the stock price low, allowing for greater liquidity (i.e. more people are able to buy and trade the stock at the lower prices). In this case, since greater liquidity is a desireable quality of a stock, especially for smaller companies, the market may react to the stock split news with a short-term price rally for the stock. But the Google stock split was allegedly done for a different reason. A class action lawsuit argued that it was done to allow Google's top executives to retain majority voting while selling off some of their stock ([source](https://www.fool.com/investing/2017/05/10/alphabet-stock-split-will-the-google-parent-ever-s.aspx)). 

Given the large size of the company, it's unlikely that the Google stock split has significantly affected the stock price movements in the long-run. We can treat the stock price drop on the day of the split as an outlier, and exclude it from the data by dividing the pre-split prices by two.


**Let's adjust the close prices for the stock split in April 3rd, 2014**  

First, let's take a look at what the stock split looks like in the raw data.

In [267]:
#stock_data.head()
# The split for GOOGL took place on April 03, 2014. (source: https://www.stocksplithistory.com/alphabet/)
stock_data.loc[stock_data.index < '2014-04-05'].tail()

,open,high,low,close,volume,close_split_adj,volume_split_adj
date,,,,,,,
2014-03-31,1130.40,1135.0000,1112.850,1114.51,3880700.0,557.255,7761400.0
2014-04-01,1120.27,1137.5000,1118.000,1134.89,2181000.0,567.445,4362000.0
2014-04-02,1141.90,1144.8000,1124.000,1135.10,4168000.0,567.550,8336000.0
2014-04-03,573.39,588.3000,566.010,571.50,4018300.0,571.500,4018300.0
2014-04-04,578.55,579.7173,544.494,545.25,5363700.0,545.250,5363700.0


In [270]:
# Adjust the pre-split close price
stock_data["close_split_adj"] = stock_data.close
stock_data["close_split_adj"].loc[stock_data.index < '2014-04-03'] = stock_data.close / 2

# Adjust the pre-split daily trading volume
stock_data["volume_split_adj"] = stock_data.volume
stock_data["volume_split_adj"].loc[stock_data.index < '2014-04-03'] = stock_data.volume * 2

In [271]:
# Verify the adjusted values
stock_data.loc[stock_data.index < '2014-04-05'].tail()

,open,high,low,close,volume,close_split_adj,volume_split_adj
date,,,,,,,
2014-03-31,1130.40,1135.0000,1112.850,1114.51,3880700.0,557.255,7761400.0
2014-04-01,1120.27,1137.5000,1118.000,1134.89,2181000.0,567.445,4362000.0
2014-04-02,1141.90,1144.8000,1124.000,1135.10,4168000.0,567.550,8336000.0
2014-04-03,573.39,588.3000,566.010,571.50,4018300.0,571.500,4018300.0
2014-04-04,578.55,579.7173,544.494,545.25,5363700.0,545.250,5363700.0


Let's plot the adjusted and unadjusted close prices.

In [272]:
trace_close = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.close,
                    mode = "lines",
                    name = "close",
                    opacity = 0.5,
                    marker = dict(color = 'red'),
                    text= stock_data.index)

trace_close_split_adj = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.close_split_adj,
                    mode = "lines",
                    name = "split-adjusted close",
                    opacity = 0.5,
                    marker = dict(color = 'blue'),
                    text= stock_data.index)


layout = dict(title = 'GOOGL Stock Daily Close Prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Stock price'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_close, trace_close_split_adj], layout = layout)
iplot(fig)

With the split adjustment, we can see a fairly steady increasing trend in the Google stock price after the Great Recession.

In [278]:
trace_bubble = go.Scatter(
                            x = stock_data.index,
                            y = stock_data.close_split_adj,
                            mode = "markers",
                            name = "volume",
                            marker = dict(size = stock_data.volume_split_adj, 
                                          sizemode='area',
                                          sizeref=2.*max(stock_data.volume_split_adj)/(30.**2),
                                          sizemin=1,
                                          color = 'black',
                                          opacity = 0.2,
                                          line = {"width": 0}),
                            hovertext  = "close: " + stock_data.close_split_adj.astype(str) 
                                        + "<br>volume: " + (stock_data.volume_split_adj/1000000).round(2).astype(str) + "M"
                                        + "<br>date: " + stock_data.index,
                            hoverinfo = 'text'
                        )


layout = dict(title = 'GOOGL stock daily close prices and volumes, split-adjusted' 
                    + '<br><br><span style="font-size: 16px; color: darkgrey">Bubble size = Daily volume</span>',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Price'),
              autosize=False,
              width=980,
              height=700
             )
fig = dict(data = [trace_bubble], layout = layout)
iplot(fig)

## 3. Machine learning with LSTM

In [280]:
stock_data.sort_index(inplace=True)

In [281]:
stock_data.head(20)

,open,high,low,close,volume,close_split_adj,volume_split_adj
date,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,50.1675,89318000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,54.1550,45668600.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,54.7000,36512200.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,52.4350,30494600.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,53.0000,18377200.0
2004-08-26,104.95,107.95,104.66,107.910,7094800.0,53.9550,14189600.0
2004-08-27,108.10,108.62,105.69,106.150,6211700.0,53.0750,12423400.0
2004-08-30,105.28,105.49,102.01,102.010,5196700.0,51.0050,10393400.0
2004-08-31,102.32,103.71,102.16,102.370,4917800.0,51.1850,9835600.0


In [282]:
close_prices = stock_data.reset_index().close.copy()
close_prices.head()

0    100.335
1    108.310
2    109.400
3    104.870
4    106.000
Name: close, dtype: float64

#### Split the close price time series into windows

In [219]:
window_length = 30
windows = []
for i in range(len(close_prices) - (window_length)):
    window_slice = close_prices[i: i + window_length].copy().reset_index(drop=True)
    windows.append(window_slice)

In [220]:
len(windows)

3661

In [221]:
len(close_prices)

3691

#### Normalize the windows

In [222]:
normalized_windows = []
for window in windows:
    normalized_window = [( ( j - window[0] )/ window[0] ) for j in window]
    normalized_windows.append(normalized_window)
        

#### Split the data into train and test sets

In [223]:
normalized_windows = np.array(normalized_windows)

In [224]:
normalized_windows.shape

(3661, 30)

In [225]:
# Setting a 90%/10% train/test split
ntrainrows = round(len(normalized_windows) * 0.9)

train = normalized_windows[:ntrainrows, :]
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]

x_test = normalized_windows[ntrainrows:, :-1]
y_test = normalized_windows[ntrainrows:, -1]

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) 

In [226]:
len(normalized_windows) - ntrainrows

366

#### Build an LSTM model

In [227]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [228]:
# The Sequential model is a linear stack of layers.
# We can create it layer by layer.
model = Sequential()


model.add(LSTM(
    input_dim=1,
    output_dim=30,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))

model.add(Activation('linear'))

model.compile(loss='mse', optimizer='rmsprop')

#### Train the model

In [229]:
model.fit(
    x_train,
    y_train,
    batch_size=512,
    nb_epoch=1,
    validation_split=0.05)

Train on 3130 samples, validate on 165 samples
Epoch 1/1
3130/3130 [==============================] - 5s 2ms/step - loss: 0.0069 - val_loss: 0.0028


#### Get model predictions

In [230]:
window_size = 30 
prediction_len = window_size 


prediction_seqs = []
n_prediction_seqs = round(len(x_test)/prediction_len)

for i in range(n_prediction_seqs):
    curr_frame = x_test[i*prediction_len]
    predicted = []
    for j in range(prediction_len):
        predict_next_day = model.predict(curr_frame[np.newaxis,:,:])[0,0]
        predicted.append(predict_next_day)
        
        curr_frame = curr_frame[1:]
        curr_frame = np.append(curr_frame, [[predict_next_day]], axis=0)
        
    prediction_seqs.append(predicted)

In [231]:
y_test.shape

(366,)

In [232]:
# leftover days after the last prediction interval
y_test.shape[0] % n_prediction_seqs

6

#### Visualize the predictions

In [233]:
dates_test = stock_data.index[-len(y_test):]

# Predicted normalized y values
y_predicted_normalized = []
for seq in prediction_seqs:
    for p in seq:
        y_predicted_normalized.append(p)

In [234]:
len(prediction_seqs)

12

In [235]:
# Normalized test data (observed y values)
trace_close_norm = go.Scatter(
                    x = dates_test,
                    y = y_test,
                    mode = "markers",
                    name = "observed",
                    marker = dict(color = 'grey'),
                    text = dates_test )

trace_predicted_norm = go.Scatter(
                    x = dates_test,
                    y = y_predicted_normalized,
                    mode = "markers",
                    name = "predicted",
                    marker = dict(color = 'blue'),
                    text = dates_test )


layout = dict(title = 'Normalized 30-day window changes in GOOGL Stock Daily Prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Stock price changes'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_close_norm, trace_predicted_norm], layout = layout)
iplot(fig)

In [236]:
x_test.shape

(366, 29, 1)

In [237]:
leftover_days = y_test.shape[0] % n_prediction_seqs

windows_test = np.array(windows)[ntrainrows:-leftover_days, :-1]
x_test_starts = [] 
for i in range(len(windows_test)):
    x_test_starts.append(windows_test[i][0])

In [238]:
len(x_test_starts)

360

In [239]:
len(y_predicted_normalized)

360

In [240]:
# Denormalized predicted y values
y_predicted_denormalized = []
for i in range(len(y_predicted_normalized)):
    denorm = (y_predicted_normalized[i] + 1) * x_test_starts[i]
    y_predicted_denormalized.append(denorm)

In [241]:
y_predicted_denormalized[:10]

[1017.5248215849697,
 1019.944560810551,
 1017.7732564504445,
 1010.5414330163597,
 1016.4474114386737,
 1043.3908076547086,
 1051.3268297369777,
 1063.6938023793698,
 1059.5418408521266,
 1067.333697924018]

In [242]:
# Normalized test data (observed y values)
trace_close_test = go.Scatter(
                    x = dates_test,
                    y = stock_data.close[-len(y_test):],
                    mode = "lines",
                    name = "observed",
                    marker = dict(color = 'grey'),
                    text = dates_test )

trace_predicted_denorm = go.Scatter(
                    x = dates_test,
                    y = y_predicted_denormalized,
                    mode = "lines",
                    name = "predicted",
                    marker = dict(color = 'blue'),
                    text = dates_test )


layout = dict(title = 'Predicted and observed GOOGL Stock Daily Prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Stock price'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_close_test, trace_predicted_denorm], layout = layout)
iplot(fig)

#### Model performance evaluation

In [243]:
# Mean squared error of the predicted normalized movements of the daily close price
mse_norm = np.mean((y_test[:-6] - y_predicted_normalized)**2)
mse_norm

0.008387967094235775

In [244]:
# Mean squared error of the predicted daily close price
mse_denorm = np.mean((stock_data.close[-len(y_test):-6] - y_predicted_denormalized)**2)
mse_denorm

10613.014075099474

In [249]:
# use the split-adjusted close price
close_split_adj_prices = stock_data.reset_index().close_split_adj.copy()

In [250]:
close_split_adj_prices.head()

0    50.1675
1    54.1550
2    54.7000
3    52.4350
4    53.0000
Name: close_split_adj, dtype: float64

In [253]:
close_prices_split_adj = stock_data.reset_index().close_split_adj.copy()

In [254]:
window_length = 30
windows = []
for i in range(len(close_prices_split_adj) - (window_length)):
    window_slice = close_prices_split_adj[i: i + window_length].copy().reset_index(drop=True)
    windows.append(window_slice)

In [255]:
# normalize the windows
normalized_windows = []
for window in windows:
    normalized_window = [( ( j - window[0] )/ window[0] ) for j in window]
    normalized_windows.append(normalized_window)

In [256]:
# split the data into train and test 
normalized_windows = np.array(normalized_windows)


# Setting a 90%/10% train/test split
ntrainrows = round(len(normalized_windows) * 0.9)

train = normalized_windows[:ntrainrows, :]
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]

x_test = normalized_windows[ntrainrows:, :-1]
y_test = normalized_windows[ntrainrows:, -1]

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) 

In [257]:
# The Sequential model is a linear stack of layers.
# We can create it layer by layer.
model2 = Sequential()


model2.add(LSTM(
    input_dim=1,
    output_dim=30,
    return_sequences=True))
model2.add(Dropout(0.2))

model2.add(LSTM(
    100,
    return_sequences=False))
model2.add(Dropout(0.2))

model2.add(Dense(
    output_dim=1))

model2.add(Activation('linear'))

model2.compile(loss='mse', optimizer='rmsprop')

In [258]:
# train the model
model2.fit(
    x_train,
    y_train,
    batch_size=512,
    nb_epoch=1,
    validation_split=0.05)

Train on 3130 samples, validate on 165 samples
Epoch 1/1
3130/3130 [==============================] - 5s 2ms/step - loss: 0.0067 - val_loss: 0.0024


In [259]:
# get predictions 

window_size = 30 
prediction_len = window_size 


prediction_seqs = []
n_prediction_seqs = round(len(x_test)/prediction_len)

for i in range(n_prediction_seqs):
    curr_frame = x_test[i*prediction_len]
    predicted = []
    for j in range(prediction_len):
        predict_next_day = model.predict(curr_frame[np.newaxis,:,:])[0,0]
        predicted.append(predict_next_day)
        
        curr_frame = curr_frame[1:]
        curr_frame = np.append(curr_frame, [[predict_next_day]], axis=0)
        
    prediction_seqs.append(predicted)

In [260]:
# visualize predictions
dates_test = stock_data.index[-len(y_test):]

# Predicted normalized y values
y_predicted_normalized = []
for seq in prediction_seqs:
    for p in seq:
        y_predicted_normalized.append(p)

In [261]:
# Normalized test data (observed y values)
trace_close_norm = go.Scatter(
                    x = dates_test,
                    y = y_test,
                    mode = "markers",
                    name = "observed",
                    marker = dict(color = 'grey'),
                    text = dates_test )

trace_predicted_norm = go.Scatter(
                    x = dates_test,
                    y = y_predicted_normalized,
                    mode = "markers",
                    name = "predicted",
                    marker = dict(color = 'blue'),
                    text = dates_test )


layout = dict(title = 'Normalized 30-day window changes in GOOGL Stock Daily Prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Stock price changes'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_close_norm, trace_predicted_norm], layout = layout)
iplot(fig)

In [262]:
leftover_days = y_test.shape[0] % n_prediction_seqs

windows_test = np.array(windows)[ntrainrows:-leftover_days, :-1]
x_test_starts = [] 
for i in range(len(windows_test)):
    x_test_starts.append(windows_test[i][0])

In [263]:
# Denormalized predicted y values
y_predicted_denormalized = []
for i in range(len(y_predicted_normalized)):
    denorm = (y_predicted_normalized[i] + 1) * x_test_starts[i]
    y_predicted_denormalized.append(denorm)

In [264]:
# Normalized test data (observed y values)
trace_close_test = go.Scatter(
                    x = dates_test,
                    y = stock_data.close[-len(y_test):],
                    mode = "lines",
                    name = "observed",
                    marker = dict(color = 'grey'),
                    text = dates_test )

trace_predicted_denorm = go.Scatter(
                    x = dates_test,
                    y = y_predicted_denormalized,
                    mode = "lines",
                    name = "predicted",
                    marker = dict(color = 'blue'),
                    text = dates_test )


layout = dict(title = 'Predicted and observed GOOGL Stock Daily Prices',
              xaxis= dict(zeroline= False),
              yaxis= dict(title= 'Stock price'),
              autosize=False,
              width=980,
              height=600
             )
fig = dict(data = [trace_close_test, trace_predicted_denorm], layout = layout)
iplot(fig)

In [265]:
# Model performance evaluation

# Mean squared error of the predicted normalized movements of the daily close price
mse_norm = np.mean((y_test[:-6] - y_predicted_normalized)**2)
mse_norm

0.008387967094235775

In [266]:
# Mean squared error of the predicted daily close price
mse_denorm = np.mean((stock_data.close[-len(y_test):-6] - y_predicted_denormalized)**2)
mse_denorm

10613.014075099474

### Comparing to Moving Average model